In [ ]:
import os
import snowflake.connector
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
class SnowflakeTableCreator:
    def __init__(self):
        """
        Inicializa a classe SnowflakeTableCreator com variáveis de ambiente que armazenam
        as credenciais e detalhes de conexão com o Snowflake, como usuário, senha, conta,
        warehouse, banco de dados e schema. Também define as propriedades para conexão e cursor.
        """
        self.user = os.getenv("SNOWFLAKE_USER")
        self.password = os.getenv("SNOWFLAKE_PASSWORD")
        self.account = os.getenv("SNOWFLAKE_ACCOUNT_NAME")
        self.warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
        self.database = os.getenv("SNOWFLAKE_DATABASE")
        self.schema = os.getenv("SNOWFLAKE_SCHEMA")        
        self.connection = None
        self.cursor = None

    def connect(self):
        """
        Estabelece a conexão com o banco de dados Snowflake utilizando as credenciais e detalhes
        de conexão fornecidos durante a inicialização. Armazena a conexão e o cursor para executar
        comandos SQL subsequentes.
        """
        self.connection = snowflake.connector.connect(
            user=self.user,
            password=self.password,
            account=self.account,
            warehouse=self.warehouse,
            database=self.database,
            schema=self.schema
        )
        self.cursor = self.connection.cursor()

    def disconnect(self):
        """
        Fecha a conexão e o cursor com o banco de dados Snowflake, caso estejam ativos.
        Garante que os recursos sejam liberados após o uso da conexão.
        """
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()

    def create_table(self, table: str, create_table_sql: str):
        """
        Cria uma tabela no banco de dados Snowflake utilizando a instrução SQL fornecida. 
        Se a conexão não estiver ativa, levanta um erro. Em caso de sucesso, confirma a criação 
        da tabela e imprime uma mensagem. Em caso de erro, imprime a mensagem de erro e realiza
        rollback para desfazer qualquer alteração.
        
        :param table: Nome da tabela a ser criada.
        :param create_table_sql: Instrução SQL para criar a tabela no Snowflake.
        """
        if not self.connection or not self.cursor:
            raise RuntimeError("Conexão com Snowflake não estabelecida. Chame o método 'connect' primeiro.")
                
        try:
            self.cursor.execute(create_table_sql)
            print(f"Tabela {table} criada com sucesso.")
        except Exception as e:
            print(f"Erro ao criar tabela {table}: {e}")
            self.connection.rollback()